<a href="https://colab.research.google.com/github/ryogrid/ryogridJupyterNotebooks/blob/master/gpt_index_trial_changed_chunk_len_230611_2202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/gpt-index-trial

/content/drive/MyDrive/gpt-index-trial


In [4]:
!cat requirements.txt

llama-index
langchain
transformers
sentence-transformers
sentencepiece

In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5

In [6]:
from llama_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [7]:
!pwd
!ls

/content/drive/MyDrive/gpt-index-trial
requirements.txt


In [8]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_msg}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

In [9]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
qa_model = AutoModelForCausalLM.from_pretrained("oshizo/qa-refine-japanese-gpt-1b")

if torch.cuda.is_available():
    qa_model = qa_model.to("cuda")


In [10]:
def generate(prompt):

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [11]:
class CustomLLM(LLM):
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return generate(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm=llm)
# llm_predictor = LLMPredictor(llm)

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"))
embed_model._langchain_embedding.client.max_seq_length = 256

In [13]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [14]:
# OpenAIのAPIは呼び出さないがapi keyが設定されていないとインデックス作成時にエラーになる
# ため、空文字を設定している
! export OPENAI_API_KEY=""

In [15]:
from typing import Dict
from llama_index.callbacks.base import BaseCallbackHandler
from llama_index.callbacks.schema import CBEventType

class CustomCallbackHandler(BaseCallbackHandler):

    def __init__(
        self,
        event_starts_to_ignore: Optional[List[CBEventType]] = None,
        event_ends_to_ignore: Optional[List[CBEventType]] = None,
        print_trace_on_end: bool = True,
    ) -> None:
        event_starts_to_ignore = event_starts_to_ignore if event_starts_to_ignore else []
        event_ends_to_ignore = event_ends_to_ignore if event_ends_to_ignore else []
        super().__init__(
            event_starts_to_ignore=event_starts_to_ignore,
            event_ends_to_ignore=event_ends_to_ignore,
        )

    def on_event_start(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> str:
        print(f"event_type = {event_type} (start)")
        return event_id

    def on_event_end(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> None:
        print(f"event_type = {event_type} (end)")
        if event_type == CBEventType.RETRIEVE:
          for selected_node in payload["nodes"]:
            print(selected_node)

    def start_trace(self, trace_id: Optional[str] = None) -> None:
        print("start_trace")

    def end_trace(
        self,
        trace_id: Optional[str] = None,
        trace_map: Optional[Dict[str, List[str]]] = None,
    ) -> None:
        print("end_trace")

In [37]:
#from llama_index import GPTVectorStoreIndex
from llama_index import LLMPredictor, ServiceContext, GPTSimpleKeywordTableIndex, VectorStoreIndex
from llama_index.readers.schema.base import Document
from llama_index.indices.list.base import ListRetrieverMode
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.indices.service_context import set_global_service_context
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.response.type import ResponseMode
from llama_index.indices.prompt_helper import PromptHelper

documents = []
chunkLen = 600 #200
for i in range(0, len(text), chunkLen):
    documents.append(Document(text[i:i+chunkLen]))
    if i != 0:
        documents.append(Document(text[i-int(chunkLen/2):i+int(chunkLen/2)]))

#llm_predictor   = LLMPredictor(llm=llm)
llama_debug_handler = LlamaDebugHandler()
custom_callback_handler = CustomCallbackHandler()
callback_manager = CallbackManager([
    llama_debug_handler
    , custom_callback_handler
])
#callback_manager = CallbackManager([llama_debug_handler])

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager, embed_model=embed_model, prompt_helper=PromptHelper.from_llm_metadata(llm_predictor.get_llm_metadata()))
#set_global_service_context(service_context=service_context)
#service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# インデックスの作成
#index = GPTVectorStoreIndex.from_documents(documents, llm_predictor=llm_predictor, embed_model=embed_model)
#index = GPTSimpleKeywordTableIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)

start_trace
event_type = node_parsing (start)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (sta

In [38]:
len(documents)

85

In [18]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [39]:
#query_engine = index.as_query_engine()

'''
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=2,
)
'''
retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=5)
#retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=10)
#retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=20)
#retriever = index.as_retriever(retriever_mode='embedding')

response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.REFINE ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.COMPACT ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.SIMPLE_SUMMARIZE ,service_context=service_context, text_qa_template=default_prompt, callback_manager=callback_manager, verbose=True)
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer, callback_manager=callback_manager)
'''
query_engine = index.as_query_engine(
    retriever_mode=ListRetrieverMode.EMBEDDING
    , similarity_top_k=3
)
'''

response = query_engine.query(
    "虹夏ちゃんのお姉さんの仕事は？"
)
'''
response = index.query(
    "虹夏ちゃんのお姉さんの仕事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''
response.response

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='へと呼び方を改める。同じ学校のひとりのことを気にかけるが、一方でひとりの性格などを歯に衣着せず述べ、無自覚のうちに精神的ダメージを与えることもしばしば。そのストレートな物言いは虹夏（アニメではリョウ）に「ナチュラルに鬼畜」と評された。両親は共に公務員であり、曰く「母親はしっかりしている」とのこと。 伊地知 星歌（いじち せいか） 声 - 内田真礼 虹夏の姉で、ライブハウス「STARRY」の店長。クリスマスイブが誕生日で、初登場時29歳。作中1年目に30歳を迎え、2年目に31歳を迎えた。かつては自身もバンドマン（ギタリスト）で、その実力はレーベルからスカウトの話が来るほどだった。口では虹夏や結束バンドに厳しくあたることもあるが、実際には誰よりも結束バンドのことを親身に考えており、特に虹夏の事で一喜一憂する。ぶっきらぼうな態度や性格に似合わず可愛い物が好きで、ぬいぐるみを抱いていたりメイド服や女子学生服などを鑑賞用に所持していたりする。また涙もろい。 ひとりのギターヒーローとしての正体まではわからなかったものの、ひとりのギタリストとしての才能にもいち早く気づいており、ひとりを中心にした結束バンドの動画を密かに撮影するなど注目している。 PAさん 声 - 小岩井ことり ミステリアスな風貌をしたSTARRYの女性PAエンジニア。最終学歴は高校中退。プライベートでは「音戯 アルト（おとぎ ア', doc_id='59842dcb-540c-4bcb-ac9c-3ae7218414c1', embedding=None, doc_hash='c274d6bcc779dd61fd460f8e96b2ea0ec35842787b7f24e5dae78d43241fd0b2', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'ライブハウス「STARRY」の店長'

In [40]:
response = query_engine.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
)

response.response

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='ているといち早く気付いている。 先述の性格に加え、運動や勉強など取り柄と言えるものがないというコンプレックスを持っていたところ、中学1年時に暗い学生時代から一転してスターとなったバンドマンのインタビューを目にしたことで、父親の直樹から借りたギターに没頭する。高校入学後はギターを持って公園にいたところを虹夏から声を掛けられ、逃げた郁代の代わりとして臨時で結束バンドのギターを務める運びになり、その流れで結束バンドに加入した。その後は改めて加入した郁代とギターの指導による師弟関係を結び、また虹夏やリョウとの交流を経て当初は自分のためとしてやっていた音楽の道を、結束バンドの皆と一緒に進みたいと思えるようになった。また、きくりとの出会いや様々なライブを経て、人間的に成長していく。 毎日6時間以上ギターの練習を約3年間欠かさずやってきたため、プロレベルと称されるほど個人としての演奏技術はずば抜けて高い。一方で人前が苦手な性格や、他人と合わせながら演奏する経験の不足からバンド演奏では実力を十分に発揮できず、時折その片鱗を覗かせるに留まっている。結束バンドでは作詞を請け負っているものの、青春に関連したNGワードを避けており自ずと暗めの歌詞になる。レーベルミニアルバム制作時には作曲・編曲にも挑戦したものの、編曲の方がうまくいかずリョウにより遠回しに遠ざけられ、結局ひとりが作曲したものをリョウが編曲す', doc_id='ccfde95e-a03d-41a3-b812-9788224664f9', embedding=None, doc_hash='ac089af4a61ee2b844b86d6cecae2e69d60d617f9b312a754418d5ce54fead57', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'郁代'

In [41]:
response = query_engine.query(
    "ギターヒーローの正体は誰？", 
)
response.response


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングルから撮影された人気バンドのカバー動画を投稿しており、投稿サイトでの登録者数は8万人近く。ぽいずん♡やみのような音楽関係者からも注目を集めている。動画は直樹によって収益化されており、文化祭ライブで壊れたギターを新調する費用等に充てられた。 ライブでの現在の自身の実力ではギターヒーローの名前でファンを集めても満足させるだけの演奏ができないというひとりの考えから、正体が「結束バンドの後藤ひとり」であることは公にはされていない。 伊地知 虹夏（いじち にじか） 声 - 鈴代紗弓 誕生日：5月29日 / 身長：154cm / 体重：48kg / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話焼きなバンドのリーダーでまとめ役。また、本作におけるツッコミ役。水玉のリボンがトレードマークで、たいてい身体のどこかに身につけている。姉の星歌がライブハウスの店長を務める元バンドマンであることの影響か', doc_id='af746884-c50c-4e9a-a003-9b51400af7b2', embedding=None, doc_hash='fa838b7cb0eebc5f935b838872a763bd2b02d6f50ad79f2e134a84388afe57f6', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'後藤ひとり'

In [44]:
response = query_engine.query(
    "喜多郁代の両親の仕事は？", 
)
response.response

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='にも採用されたことがあるが、本人曰く「父さんのバンドはそこがピークでその後は鳴かず飛ばず」のちに妻となる美智代に「就職しなかったら別れる」と言われて辞めたという生々しい経験を語って聞かせた。 本人には教えていなかったが、ひとりの動画投稿サイトでの活動を収益化する手続きをしており（実在の動画サイトであれば本人が18歳以上であるか法定後見人が必要となる）、ひとりがギターを壊してしまった際に広告収入で得たお金を与えている。 目はいつも何か（髪の毛や、ふたりの手など）に隠されていて描かれていない。 会社では窓際族と説明しているが、自宅は金沢八景（横浜市）の戸建てである。 後藤 美智代（ごとう みちよ） 声 - 末柄里恵 誕生日：4月3日 /血液型：A型 後藤姉妹の母。ひとり及びふたりと同じくピンク色の髪をしている。人見知りなひとりを何かと心配している。喜多が家に来たときは場を盛り上げようと、喜んで学生コスプレまでした（21年ぶりに制服を着たとのこと）。結束バンドを夫と一緒になって応援しており、そのためにも学生に扮したことがある。 後藤 ふたり（ごとう ふたり） 声 - 和多田美咲 誕生日：7月7日 /血液型：A型 ひとりの妹。5歳。性格は姉とは正反対で初対面の相手にも物怖じしない。無邪気に相手の心を抉るようなことを言って、（特にひとりに）精神的ダメージを与えることが多い。 ジミヘン 声 -', doc_id='f2bc71ff-61af-444a-b4ac-a6d1f4e2891b', embedding=None, doc_hash='e81dc76a5227be78142b88580d038f72a8e850d4c4aeb90fe87bc5b938c98e0a', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'レコーディングエンジニア'

In [43]:
response = query_engine.query(
    "ひとりが動画投稿する時のハンドルネームは？", 
)
response.response

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングルから撮影された人気バンドのカバー動画を投稿しており、投稿サイトでの登録者数は8万人近く。ぽいずん♡やみのような音楽関係者からも注目を集めている。動画は直樹によって収益化されており、文化祭ライブで壊れたギターを新調する費用等に充てられた。 ライブでの現在の自身の実力ではギターヒーローの名前でファンを集めても満足させるだけの演奏ができないというひとりの考えから、正体が「結束バンドの後藤ひとり」であることは公にはされていない。 伊地知 虹夏（いじち にじか） 声 - 鈴代紗弓 誕生日：5月29日 / 身長：154cm / 体重：48kg / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話焼きなバンドのリーダーでまとめ役。また、本作におけるツッコミ役。水玉のリボンがトレードマークで、たいてい身体のどこかに身につけている。姉の星歌がライブハウスの店長を務める元バンドマンであることの影響か', doc_id='af746884-c50c-4e9a-a003-9b51400af7b2', embedding=None, doc_hash='fa838b7cb0eebc5f935b838872a763bd2b02d6f50ad79f2e134a84388afe57f6', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'「もっとガチにした方がいい」'

In [45]:
response = query_engine.query(
    "後藤 ひとりのニックネームは？" 
)
response.response

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='は虹夏の希望（未定）で彼女は作詞作曲には関わっていないため、実際のジャンルは不明である。 ^ 後藤正文の12月2日→後藤ひとりの2月21日、伊地知潔の9月25日→伊地知虹夏の5月29日、山田貴洋の8月19日→山田リョウの9月18日、喜多建介の1月24日→喜多郁代の4月21日。 ^ 本人は他人から初めて名付けてもらったあだ名と喜び、気に入っている。この名を付けられたきっかけは、ライブでのメンバー紹介時に本名を出してほしくないと希望したため。以前は虹夏から「ひとりちゃん」、リョウから「ひとり」と呼ばれていた。きくりもニックネームを知るまでは「ひとりちゃん」と呼んでいた。 ^ テレビ番組『69号室の住人』の「ぼっち・ざ・ろっく! 特集」における、担当編集の瀬古口拓也の発言より。 ^ 親には可愛い服も買ってもらっているが、本人の好みではなく着ていない（その服は喜多が借りたこともある）。稀に自分で服などを選んでも、バンドメンバーにセンスを酷評されている。 ^ ただし結束バンド加入後はロインで誘われると家から出たり、夏休みはメンバーから遊びに誘われるのを期待して予定を空けておく（実際は入れる予定がそもそもない）など、少しずつではあるが変わりつつある。 ^ プロのギタリストになってテレビでインタビューされた時には何を答えるか考える、逆に人生へ失敗してニートに落ちぶれた自分を想像するなど。 ^ バ', doc_id='e6427935-f5fd-4572-9d4b-6f323e909fad', embedding=None, doc_hash='35abc958c90e4b79371c829f29e33486dccfe72c5e3a0b05d91a2efccc8005d6', extra_info=None, node_info={'start': 0, 'end': 600, '_node_type': <Nod

'リョウ'